# Building a Module

A module in Python is conceptually not much more than a file with python code in it that you can access and run by importing the module.

When you `import` a module, you are reading the file and making the code contained in it accessible through its alias.

Start by creating a new file called `hello.py` and add the following message in there:

```python
message = 'Hello Jim'
```

and then try executing this code:

```python
import hello as h
h.message
```

Great!

Now, let's change the message. Edit the file to now say:

```python
message = 'Hello Jane'
```

and rexecute the code:

```python
import hello as h
h.message
```

What went wrong?

Since the contents of modules dont usually change very much (except during module development) Python is smart about importing modules and once a module is imported, it doesnt bother re-importing the module. Normally, this is what you want, but in our case when we are developing the module we want to force the module to geet automatically reloaded everytime it changes.

Fortunately, there is a _magic_ command sequence that does exactly that. First, we have to load an extension called `autoreload` by executing the _magic command_ `%load_ext autoreload`. Once you have loaded that extension, you now have access to a new _magic command_ called `%autoreload` which supports different modes of autoreload. The mode we want for now is to automatically reload anything that changes, which is mode 2. So, you need to execute:

```python
%load_ext autoreload
%autoreload 2
```

Now try executing:

```python
h.message
```

and you will see that future edits to the hello file will immediately be reloaded.

Try editing the `hello.py` file to:

```python
message = 'Hello John and Jane'
```

and then execute

```python
h.message
```

Now, let's create our new module, which we'll build on through the course, and put the `drawdown` function we created in that module. Create a file called `edhec_risk_kit.py` and copy the follwing code into it.

```python

import pandas as pd

def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 10', 'Hi 10']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets

```


In [14]:
#see top of page for instructions
#all info being pulled from our modules 
#make sure to use autoreloader if changes are made to any of those files
#hello.py and edhec_risk_kit.py
import Hello as h
print(h)

<module 'Hello' from '/storage/emulated/0/Homework/Hello.py'>


In [17]:
h.message

'Hello Jacci'

In [35]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
%autoreload 2

In [40]:
import edhec_risk_kit as erk

In [44]:
returns = erk.get_ffme_returns()
returns.describe()

,SmallCap,LargeCap
count,1110.000000,1110.000000
mean,0.017598,0.008870
std,0.106288,0.053900
min,-0.291400,-0.284300
25%,-0.027350,-0.017025
50%,0.010450,0.011400
75%,0.049950,0.037650
max,1.342900,0.377900


In [60]:
returns.head()

,SmallCap,LargeCap
1926-07,-0.0145,0.0329
1926-08,0.0512,0.0370
1926-09,0.0093,0.0067
1926-10,-0.0484,-0.0243
1926-11,-0.0078,0.0270


In [49]:
down= erk.drawdown(returns['SmallCap'])

In [50]:
down.head()

,Wealth,Previous Peak,Drawdown
1926-07,985.500000,985.500000,0.000000
1926-08,1035.957600,1035.957600,0.000000
1926-09,1045.592006,1045.592006,0.000000
1926-10,994.985353,1045.592006,-0.048400
1926-11,987.224467,1045.592006,-0.055822


In [56]:
down['Drawdown'].min()

-0.8330007793945303

In [59]:
down['Wealth'].std()

535679235.0920746

In [61]:
returns['LargeCap'].head()

1926-07    0.0329
1926-08    0.0370
1926-09    0.0067
1926-10   -0.0243
1926-11    0.0270
Freq: M, Name: LargeCap, dtype: float64